In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import os 

os.environ.update({"ALLENNLP_CACHE_ROOT": "/brtx/603-nvme2/estengel/annotator_uncertainty/vqa/"})

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp")


In [4]:
import json 

data_path = "../../hit3.0/results/json_data/dev.jsonl"
with open(data_path) as f1:
    data = [json.loads(line) for line in f1]


In [25]:
prompt = "Rephrase the following question to match the answer.\nQuestion: {question}\nAnswer: {answer}\nNew Question: "

all_prompts = []
for example in data:
    old_question = example['original_question']
    for annotation in example["annotations"]:

        clusters_and_questions = zip(annotation['new_clusters'], annotation['new_questions'])
        for c, q in clusters_and_questions:
            for answer_dict in c:
                answer = answer_dict['content']
                filled_prompt = prompt.format(question=old_question, answer=answer)
                all_prompts.append(filled_prompt)


In [26]:
for prompt in all_prompts:
    print(prompt)
    tokenized_prompt = tokenizer(prompt, return_tensors="pt")
    new_question = model.generate(**tokenized_prompt, do_sample=True, max_length=50, temperature=0.0001)
    print(tokenizer.batch_decode(new_question))
    print()

Rephrase the following question to match the answer.
Question: What are the pizzas on?
Answer: plates
New Question: 
['<pad> The pizzas are on what?</s>']

Rephrase the following question to match the answer.
Question: What are the pizzas on?
Answer: plate
New Question: 
['<pad> The pizzas are on a _.</s>']

Rephrase the following question to match the answer.
Question: What are the pizzas on?
Answer: metal pan
New Question: 
['<pad> The pizzas are on what?</s>']

Rephrase the following question to match the answer.
Question: What are the pizzas on?
Answer: tray
New Question: 
['<pad> The pizzas are on a what?</s>']

Rephrase the following question to match the answer.
Question: What are the pizzas on?
Answer: pan
New Question: 
['<pad> The pizzas are on what?</s>']

Rephrase the following question to match the answer.
Question: What are the pizzas on?
Answer: tray
New Question: 
['<pad> The pizzas are on a what?</s>']

Rephrase the following question to match the answer.
Question: Wha